In [1]:
import pandas as pd
import plotly.graph_objects as go

In [2]:
ma_test_res = pd.read_csv("ma_test_res.csv")
all_trades = pd.read_csv('all_trades.csv')

In [3]:
all_trades.shape
ma_test_res.shape

(18, 9)

In [5]:
ma_test_res = ma_test_res[['pair', 'num_trades', 'total_gain', 'mashort', 'malong']]
#ma_test_res = ma_test_res[['num_trades', 'total_gain', 'mashort', 'malong']]

In [6]:
ma_test_res['CROSS'] = "MA_" + ma_test_res.mashort.map(str) + "_" + ma_test_res.malong.map(str) #Helps to group MA to analyze

In [7]:
df_all_gains = ma_test_res.groupby(by=['CROSS', 'mashort', 'malong'], as_index=False).sum()

In [8]:
df_all_gains.sort_values(by='total_gain', ascending=False, inplace=True)

In [9]:
ma_8_21 = ma_test_res[ma_test_res.CROSS=='MA_8_21'].copy()

In [10]:
ma_8_21.sort_values(by='total_gain', ascending=False, inplace=True)

In [11]:
total_p =len(ma_8_21.pair.unique())

In [12]:
ma_8_21[ma_8_21.total_gain>0].shape[0]

0

In [13]:
for cross in df_all_gains.CROSS.unique():
    df_temp = ma_test_res[ma_test_res.CROSS==cross]
    total_p = df_temp.shape[0]
    n_good = df_temp[df_temp.total_gain>0].shape[0]
    print(f'{cross:12} {n_good} {(n_good/total_p)*100:4.0f}%')
    

MA_12_21     2  100%
MA_12_34     1   50%
MA_8_34      0    0%
MA_10_21     0    0%
MA_10_34     1   50%
MA_8_21      0    0%
MA_12_55     0    0%
MA_8_55      0    0%
MA_10_55     0    0%


In [14]:
crosses = df_all_gains.CROSS.unique()

In [15]:
df_good = ma_test_res[(ma_test_res.CROSS.isin(crosses)) & (ma_test_res.total_gain>0)].copy()

In [16]:
our_pairs = list(df_good.pair.value_counts().index) #gives list how often ea pair is in df_good, aka how often sucessfull cross.

In [17]:
all_trades.head(4)

,Unnamed: 0,open_time,close,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAINS,PAIR,MASHORT,MALONG,DURATION
0,34,2025-11-01 00:34:00+00:00,109445.34,-2.017262,8.519405,-1,96.62,-96.62,BTC_USD,8,21,0.133333
1,42,2025-11-01 00:42:00+00:00,109541.96,5.583512,-3.768750,1,237.10,237.10,BTC_USD,8,21,0.566667
2,76,2025-11-01 01:16:00+00:00,109779.06,-4.064821,0.924107,-1,84.23,-84.23,BTC_USD,8,21,0.200000
3,88,2025-11-01 01:28:00+00:00,109863.29,1.317024,-12.795536,1,-80.28,-80.28,BTC_USD,8,21,0.216667


In [18]:
all_trades['CROSS'] = "MA_" + all_trades.MASHORT.map(str) + "_" + all_trades.MALONG.map(str) #Helps to group MA to analyze

In [19]:
trades_btc_usd = all_trades[(all_trades.CROSS=='MA_8_21') & (all_trades.PAIR=='BTC_USD')].copy()

In [20]:
trades_btc_usd['CUM_GAIN'] = trades_btc_usd.GAINS.cumsum()

In [21]:
def plot_line(df_plot, name):
    fig = go.Figure()
    fig.add_trace(go.Scatter(  
        x=df_plot.open_time,
        y=df_plot.CUM_GAIN,
        line=dict(width=2, color='#3d825f'), #coloring
        line_shape='spline', #makes smooth curve
        name=name,
        mode='lines'
        ))
    
    fig.update_layout(width=1100, height=450,
        margin=dict(l=15, r=15, b=10),
        font=dict(size=10, color='#e1e1e1'), 
        paper_bgcolor='#1e1e1e',
        plot_bgcolor='#1e1e1e',
        title=name
        )
    fig.update_xaxes(
        linewidth=1,
        linecolor = '#3a4a54',
        showgrid=False,
        zeroline=False
        )
    
    fig.update_yaxes(
        linewidth=1,
        linecolor = '#3a4a54',
        showgrid=False,
        zeroline=False
        )
    fig.show()

In [22]:
plot_line(trades_btc_usd, 'BTC_USD')

In [24]:
c = 'MA_8_21'
for p in our_pairs:
    temp_df = all_trades[(all_trades.CROSS==c) & (all_trades.PAIR==p)].copy()
    temp_df['CUM_GAIN'] = temp_df.GAINS.cumsum()
    plot_line(temp_df, p + '_' + c)

In [26]:
for c in crosses:
    temp_df = temp_df.groupby(by='open_time', as_index=False).sum()
    temp_df['CUM_GAIN'] = temp_df.GAINS.cumsum()
    plot_line(temp_df, c)